In [ ]:
import numpy as np
import pandas as pd
from dateutil import parser
import os
import random

**Important note**: The column use_case_id was manually added to the UCs in PostgreSQL before extracting them as CSV files

In [ ]:
# list of Use Case CSV files in the input directory
print(os.listdir('../input'))

# dictionary containing the information of the UCs with over 100k rows
# key = UC | value = n_rows
dic_UCs = {
  "l001_bb": 898147,
  "l017_bb": 304192,
  "l020": 116533,
  "l026_bb": 238681,
  "l029_bb": 7150609,
  "l029_du": 860512,
  "l032": 3063568,
  "l033": 385771,
  "l040": 111034,
  "l080": 151111,
  "l081": 3722470,
  "l083": 21672526,
  "l090" : 24583957
}

#### The following 2 cells were used only to get the number of rows of each input CSV file for the paper

In [ ]:
directory = '../input/'

# list with the UC DataFrames
uc_list = []

for file in os.listdir(directory):
    path = os.path.join(directory, file)
    # ler apenas os nomes das colunas do CSV para determinar as colunas selecionadas
    header_data = pd.read_csv(path, nrows=0)

    # ler apenas os nomes das colunas do CSV para determinar as colunas selecionadas
    header_data = pd.read_csv(path, nrows=0)

    uc = os.path.basename(path)

    # dictionary to store the indexes of the selected columns
    dic_selected_columns = {}
    desired_columns = ['use_case_id']

    # encontrar os índices das colunas
    for col_name in desired_columns:
        if col_name in header_data.columns:
            col_index = header_data.columns.get_loc(col_name)
            dic_selected_columns[col_name] = col_index

    col_indexes = list(dic_selected_columns.values())

    print(f'Columns selected for UC {uc}: {dic_selected_columns}')
    print(f'Indexes of selected columns: {col_indexes}')

    uc_list.append(pd.read_csv(path, usecols=col_indexes))

    print(f'Extraction of UC {uc} finished\n')

print('Extraction of all UCs finished\n')

In [ ]:
for df in uc_list:
    n_rows, n_cols = df.shape
    uc = df['use_case_id'].iloc[0]
    print(f'{uc} & {n_rows} \\\\')
    print('\\hline')

#### Dataset exploratory analysis

Convert and sort datetime columns.

Join in a single CSV file.

In [ ]:
directory = '../input/'

# list with UCs DataFrames
df_list = []

for file in os.listdir(directory):
    path = os.path.join(directory, file)

    # get UC name from CSV file name
    index_slash = path.rfind('/')
    index_dot = path.rfind('.')
    uc = ''
    if index_slash != -1 and index_dot != -1:
        uc = path[index_slash + 1:index_dot]

    # read only the header of the CSV in order to determine the selected columns
    header_data = pd.read_csv(path, nrows=0)

    # dictionary to store the índexes of desired columns
    dic_selected_cols = {}
    desired_cols = ['datetime_id', 'local_datetime', 'use_case_id']

    # set column indexes
    for col_name in desired_cols:
        if col_name in header_data.columns:
            col_index = header_data.columns.get_loc(col_name)
            dic_selected_cols[col_name] = col_index

    # if the UC has both datetime columns with and without timezones, remove local_datetime
    if 'datetime_id' in dic_selected_cols and 'local_datetime' in dic_selected_cols:
        del dic_selected_cols['local_datetime']

    # columns read and included in the DataFrame
    col_indexes = list(dic_selected_cols.values())

    print(f'Columns selected for UC {uc}: {dic_selected_cols}')
    print(f'Indexes of selected columns: {col_indexes}')
    print('Starting file read [limit of 100000 rows]...')

    # if the UC has more than 100k rows (present in dic_UCs), calculate sample intervals
    if dic_UCs.get(uc) != None:
        # total of rows in the file (minus the header)
        total_rows = dic_UCs[uc] - 1
        # define the number of rows to extract to 100k
        n_rows_to_extract = 100000
        # sample of rows to be ignored
        ignored_rows_sample = total_rows - n_rows_to_extract
        # list with the numbers of the rows to be skipped chosen randomly
        skip_rows =  random.sample(range(1, total_rows+1), ignored_rows_sample)
        print(f'Number of rows chosen randomly to be ignored: {len(skip_rows)}')
        df_list.append(pd.read_csv(path, skiprows=skip_rows, usecols=col_indexes))
    # if the file has less than 100k rows, extract all of them
    else:
        df_list.append(pd.read_csv(path, usecols=col_indexes))

    print(f'Extraction of UC {uc} finished\n')

print('Extraction of all UCs finished')

In [ ]:
# conversion of datetime columns to a numeric representation
print('Starting conversion of the datetime columns to a numeric representation...')

for df in df_list:
    uc = df['use_case_id'].iloc[0]
    df_cols = list(df.columns)
    
    if 'datetime_id' in df_cols:
        # transform datetime to numeric representation (// 10**9 to convert from nanoseconds to secondss)
        df['datetime_id'] = pd.to_datetime(df['datetime_id'], format='%Y-%m-%d %H:%M:%S').astype(np.int64) // 10**9
    
    # if the DataFrame has local_datetime column (with time zone)
    elif 'local_datetime' in df_cols:
        # convert local_datetime to timestamp
        df['local_datetime'] = df['local_datetime'].apply(lambda x: parser.parse(x))
        # create datetime_utc column with + 3 hours to be able to convert to a numeric representation
        df['datetime_utc'] = df['local_datetime'].apply(lambda x: x.tz_localize(None) + pd.Timedelta(hours=3))
        # print first row of datetime_utc column to check if it is correct (should have the same value of the original + 3 hours)
        print(df['datetime_utc'].iloc[0])
        # create datetime_id column and convert to numeric representation
        df['datetime_id'] = df['datetime_utc'].apply(lambda x: (x - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))
        # remove local_datetime and datetime_utc columns
        df = df.drop(['local_datetime', 'datetime_utc'], axis=1)

    print(f'\nConversion of datetime of UC {uc} finished')

print(f'\nConversion of all UCs datetime columns to a numerical representation finished')

In [ ]:
# set order of columns to datetime_id | use_case_id
for df in df_list:
    # if the columns are not in the correct order, invert them
    if df.columns[0] == 'use_case_id' and df.columns[1] == 'datetime_id':
        df = df.rename(columns={'use_case_id': 'temp'})
        df = df.rename(columns={'datetime_id': 'use_case_id'})
        df = df.rename(columns={'temp': 'datetime_id'})

        df['datetime_id'], df['use_case_id'] = df['use_case_id'], df['datetime_id']
        print('Column order inverted')
    
    else:
        print('Column order already as datetime_id | use_case_id, no changes')

In [ ]:
# print general information of DataFrames
for df in df_list:
    print(df.info())
    print('-'*50)

In [ ]:
# print first 2 rows of each DataFrame
for df in df_list:
    print(df.head(2))
    print('-'*50)

In [ ]:
# output temporal CSV files
for i, df in enumerate(df_list):
    file_name = df['use_case_id'].iloc[0]
    df.to_csv('../output/temporal_'+ file_name, index=False)
    print(f'"temporal_{file_name}" generated')

In [ ]:
# list of files in output folder
print(os.listdir('../output'))

After all UCs are joined into a single CSV file, it is necessary to **sort the datetime_id column in ascending order**, then the table will be compatible with statistical analysis, such as **linear correlation**.

Example of desired layout:

|datetime_id | use_case_id|

|00181       | l006       |

|00181       | l080       |

|02405       | l090       |

|02405       | l050       |

|02405       | l084       |

In [ ]:
# join all UCs
UCs_list = []
print('Joining temporal files...')

df_uc001_bb = pd.read_csv('./output/temporal_l001_bb.csv', delimiter=',')
UCs_list.append(df_uc001_bb)
df_uc006 = pd.read_csv('./output/temporal_l006.csv', delimiter=',')
UCs_list.append(df_uc006)
df_uc017_bb = pd.read_csv('./output/temporal_l017_bb.csv', delimiter=',')
UCs_list.append(df_uc017_bb)
df_uc017_du = pd.read_csv('./output/temporal_l017_du.csv', delimiter=',')
UCs_list.append(df_uc017_du)
df_uc018 = pd.read_csv('./output/temporal_l018.csv', delimiter=',')
UCs_list.append(df_uc018)
df_uc020 = pd.read_csv('./output/temporal_l020.csv', delimiter=',')
UCs_list.append(df_uc020)
df_uc026_bb = pd.read_csv('./output/temporal_l026_bb.csv', delimiter=',')
UCs_list.append(df_uc026_bb)
df_uc026_du = pd.read_csv('./output/temporal_l026_du.csv', delimiter=',')
UCs_list.append(df_uc026_du)
df_uc029_bb = pd.read_csv('./output/temporal_l029_bb.csv', delimiter=',')
UCs_list.append(df_uc029_bb)
df_uc029_du = pd.read_csv('./output/temporal_l029_du.csv', delimiter=',')
UCs_list.append(df_uc029_du)
df_uc032 = pd.read_csv('./output/temporal_l032.csv', delimiter=',')
UCs_list.append(df_uc032)
df_uc033 = pd.read_csv('./output/temporal_l033.csv', delimiter=',')
UCs_list.append(df_uc033)
df_uc037_du = pd.read_csv('./output/temporal_l037_du.csv', delimiter=',')
UCs_list.append(df_uc037_du)
df_uc040 = pd.read_csv('./output/temporal_l040.csv', delimiter=',')
UCs_list.append(df_uc040)
df_uc041 = pd.read_csv('./output/temporal_l041.csv', delimiter=',')
UCs_list.append(df_uc041)
df_uc044 = pd.read_csv('./output/temporal_l044.csv', delimiter=',')
UCs_list.append(df_uc044)
df_uc050 = pd.read_csv('./output/temporal_l050.csv', delimiter=',')
UCs_list.append(df_uc050)
df_uc080 = pd.read_csv('./output/temporal_l080.csv', delimiter=',')
UCs_list.append(df_uc080)
df_uc081 = pd.read_csv('./output/temporal_l081.csv', delimiter=',')
UCs_list.append(df_uc081)
df_uc083 = pd.read_csv('./output/temporal_l083.csv', delimiter=',')
UCs_list.append(df_uc083)
df_uc084 = pd.read_csv('./output/temporal_l084.csv', delimiter=',')
UCs_list.append(df_uc084)
df_uc090 = pd.read_csv('./output/temporal_l090.csv', delimiter=',')
UCs_list.append(df_uc090)

df_union = pd.concat(UCs_list)
n_rows, n_cols = df_union.shape
print(f'Number of UCs joined: {len(UCs_list)} | Rows: {n_rows} | Columns: {n_cols}')

In [ ]:
df_union.head(5)

In [ ]:
# sort DataFrame of joined UCs:
df_union = df_union.sort_values(by='datetime_id')
print('datetime_id column sorted in ascending order')

In [ ]:
# convert datetime_id back to timestamp format
df_union['datetime_id'] = pd.to_datetime(df_union['datetime_id'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
print('datetime_id converted back to format "Y-m-d H:M:S"')

In [ ]:
df_union.head(5)

In [ ]:
df_union.tail(5)

In [ ]:
# output of joined UCs DataFrame to a new CSV file
df_union.to_csv('../output/temporal_UCs.csv', index=False)
print(f'"temporal_UCs.csv" generated')